# Implementing a Transformer Network

In this exercise, we will develop a transformer network to perform text classification on IMDb Movie Reviews dataset. 😀

## Downloading dataset 🎥
IMDb Movie Review dataset is a binary sentiment analysis dataset consisting of 50,000 reviews from the Internet Movie Database (IMDb) labeled as positive or negative.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
texts = imdb_dataset['train']['text']
labels = imdb_dataset['train']['label']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 64
batch_size = 32

train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.4)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

print(f'Train size: {len(train_dataset)}')
print(f'Validation size: {len(val_dataset)}')
print(f'Test size: {len(test_dataset)}')

Train size: 15000
Validation size: 5000
Test size: 5000


## Transformer Implementation

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=64):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # Shape: [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # Shape: [max_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # div_term helps to scale the frequency of the sine and cosine functions.

        # Hint: Apply the sine function to even indices (0, 2, 4, ...) in each position
        pe[:, 0::2] = torch.sin(position*div_term) ###fill in the blank###
         # Hint: Apply the cosine function to odd indices (1, 3, 5, ...) in each position
        pe[:, 1::2] = torch.cos(position*div_term) ###fill in the blank###

        pe = pe.unsqueeze(0)  # Shape: [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)  # Sequence length is in the second dimension
        x = x + self.pe[:, :seq_len, :].to(x.device)  # Ensure pe is on the same device as x
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % nhead == 0, "d_model must be divisible by nhead"

        self.d_model = d_model #embedding dimension
        self.nhead = nhead #number of heads
        self.d_k = d_model // nhead ##head dimension
        self.dropout = nn.Dropout(dropout)

        # Hint: Define the linear layers to project the input for query, key, and value
        self.w_q = nn.Linear(self.d_model,self.d_model)
        self.w_k = nn.Linear(self.d_model,self.d_model)
        self.w_v = nn.Linear(self.d_model,self.d_model)

        self.w_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, q, k, v, attn_mask=None):
        # Hint: Calculate the attention scores by taking the dot product between Q and K^T
        scores = torch.matmul(q, k.transpose(2,3)) / math.sqrt(self.d_k)###fill in the blank###
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask == 0, float('-inf'))

        # Hint: Convert scores to probabilities using softmax
        attn_weights = torch.softmax(scores, dim=-1) ###fill in the blank###
        # Hint: Apply dropout to the attention weights
        attn_weights = self.dropout(attn_weights)###fill in the blank###
        return torch.matmul(attn_weights, v), attn_weights

    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None):
        batch_size = query.size(0)

        q = self.w_q(query).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2) # (batch_size, nhead, seq_len, d_k)
        k = self.w_k(key).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2) # (batch_size, nhead, seq_len, d_k)
        v = self.w_v(value).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2) # (batch_size, nhead, seq_len, d_k)

        attn_output, attn_weights = self.scaled_dot_product_attention(q, k, v, attn_mask)

        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        return self.w_o(attn_output)


class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # Hint: Apply self-attention mechanism on the src
        src2 = self.self_attn(query=src, key=src, value=src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]

        # Hint: Add residual connection followed by normalization
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        #Apply the feed-forward network
        src2 = self.linear2(self.dropout1(self.linear1(src)))

        # Hint: Add residual connection followed by normalization
        src = src+self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        self.norm = norm

    def forward(self, src, mask=None, src_key_padding_mask=None):
        output = src
        for mod in self.layers:
            output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
        if self.norm is not None:
            output = self.norm(output)
        return output

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_classes, dim_feedforward=2048, dropout=0.1, max_len=512):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_encoder_layers)
        self.fc = nn.Linear(d_model, num_classes)
        self.d_model = d_model

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        embedded = self.pos_encoder(embedded)
        transformer_out = self.transformer_encoder(embedded)
        output = transformer_out.mean(dim=1)
        return self.fc(output)

Now, let's train the model.

In [ ]:
# Function to train model
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# Function to evaluate model
def eval_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    return accuracy_score(true_labels, predictions)

In [ ]:
vocab_size = tokenizer.vocab_size
d_model = 768
nhead = 12
num_encoder_layers = 4
num_classes = 2

model = TransformerModel(vocab_size, d_model, nhead, num_encoder_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)


num_epochs = 1
for epoch in range(num_epochs):
    train_loss = train_model(model, train_dataloader, criterion, optimizer, device)
    val_accuracy = eval_model(model, val_dataloader, device)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

test_accuracy = eval_model(model, test_dataloader, device)

print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1, Train Loss: 0.6619, Val Accuracy: 0.6382
Test Accuracy: 0.6428


## Using pretrained weight for BERT
Let's load the pretrained weights from BERT and see how much the accuracy improves.

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = outputs.pooler_output
        return self.fc(output)

In [ ]:
model = BERTClassifier(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)

num_epochs = 1
for epoch in range(num_epochs):
    train_loss = train_model(model, train_dataloader, criterion, optimizer, device)
    val_accuracy = eval_model(model, val_dataloader, device)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

test_accuracy = eval_model(model, test_dataloader, device)
print(f'Test Accuracy: {test_accuracy:.4f}')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1, Train Loss: 0.4476, Val Accuracy: 0.8250
Test Accuracy: 0.8262
